# How to call functions with chat models

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [1]:
import sys
print(sys.executable)

/usr/local/opt/python@3.11/bin/python3.11


In [2]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [12]:
import json
import openai
import requests
import os
from termcolor import colored

from tenacity import retry, wait_random_exponential, stop_after_attempt


GPT_MODEL = "gpt-3.5-turbo-0613"
openai.api_key  = os.environ['OPENAI_API_KEY']

ModuleNotFoundError: No module named 'termcolor'

## Utils

In [13]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [14]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "tool": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "tool":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


## Basic Concepts

In [15]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [16]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'Sure, I can help you with that. Could you please provide me with your current city and state?'}

In [17]:
messages.append({"role": "user", "content": "I'm in Melbourne, Australia."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_nyfwzzdV5sp6JoNmM8aakvvJ',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Melbourne, Australia",\n  "format": "celsius"\n}'}}]}

In [18]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Melbourne, Australia over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'Sure, I can help you with that. Please specify the number of days you would like to forecast.'}

In [19]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_3LAaSNfvQHkulGMOAJLp3BaP',
    'type': 'function',
    'function': {'name': 'get_n_day_weather_forecast',
     'arguments': '{\n  "location": "Melbourne, Australia",\n  "format": "celsius",\n  "num_days": 5\n}'}}]},
 'logprobs': None,
 'finish_reason': 'tool_calls'}

## Forcing the use of specific functions or no function

In [20]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, 
    tools=tools, 
    # forcing model to choice a function
    tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_psrVmEHbPJ4pwqDhf8hLwNJ1',
   'type': 'function',
   'function': {'name': 'get_n_day_weather_forecast',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}]}

In [21]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_7cT62fOexHmRUCphD5HUvAKo',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}}]}

In [22]:
# Don't choice a function
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}

## Parallel Function Calling

In [23]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

chat_response.json()
assistant_message = chat_response.json()["choices"][0]["message"]['tool_calls']
assistant_message

[{'id': 'call_if6CZ08cc62Zojyy8VDNrTnf',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}'}},
 {'id': 'call_hUzc9wsRrd4qRtucVm7AHqC8',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "Glasgow", "format": "celsius", "num_days": 4}'}}]

## How to call functions with model generated arguments